# **Processamento de Linguagem Natural [2023.Q3]**
Prof. Alexandre Donizeti Alves

### **ATIVIDADE PRÁTICA 04 [Uso da API da OpenAI com técnicas de PLN]**


A **ATIVIDADE PRÁTICA 04** deve ser feita utilizando o **Google Colab** com uma conta sua vinculada ao Gmail. O link do seu notebook, armazenado no Google Drive, além do link de um repositório no GitHub e os principais resultados da atividade, devem ser enviados usando o seguinte formulário:

> https://forms.gle/GzwCq3R7ExtE9g9a8


**IMPORTANTE**: A submissão deve ser feita até o dia **26/11 (domingo)** APENAS POR UM INTEGRANTE DA EQUIPE, até às 23h59. Por favor, lembre-se de dar permissão de ACESSO IRRESTRITO para o professor da disciplina de PLN.

### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:** Eduardo Renesto Estanquiere - 11201810086

### **LIVRO**
---

`Processamento de Linguagem Natural - Conceitos, Técnicas e Aplicações em Português.`

>

Disponível gratuitamente em:
  
  > https://brasileiraspln.com/livro-pln/1a-edicao/.


**POR FAVOR, PREENCHER OS CAPITULOS SELECIONADOS PARA A SUA EQUIPE:**

`Primeiro capítulo: 12`

`Segundo capítulo: 14`

### **DESCRIÇÃO**
---

Implementar um `notebook` no `Google Colab` que faça uso da **API da OpenAI** aplicando, no mínimo, 3 técnicas de PLN. As técnicas devem ser aplicadas nos 2 (DOIS) capítulos do livro **Processamento de Linguagem Natural - Conceitos, Técnicas e Aplicações em Português**.

>

**RESTRIÇÃO**: É obrigatório usar o *endpoint* "*`Chat Completions`*".

>

As seguintes técnicas de PLN podem ser usadas:

*   Correção Gramatical
*   Classificação de Textos
*   Análise de Sentimentos
*   Detecção de Emoções
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos
*   **Similaridade de Textos**
*   **Reconhecimento de Entidades Nomeadas**
*   **Sistemas de Perguntas e Respostas**

>

Os capítulos devem ser os mesmos selecionados na **ATIVIDADE PRÁTICA 02**. Para consultar os capítulos, considere a seguinte planilha:

>

> https://docs.google.com/spreadsheets/d/1ZutzQ3v1OJgsgzCvCwxXlRIQ3ChXNlHNvB63JQvYsbo/edit?usp=sharing

>
>

**IMPORTANTE:** É obrigatório usar o e-mail da UFABC. Não é permitido alterar os capítulos já selecionados.



### **CRITÉRIOS DE AVALIAÇÃO**
---


Serão considerados como critérios de avaliação as técnicas usadas e a criatividade envolvida na aplicação das mesmas.




### **IMPLEMENTAÇÃO**
---

Antes de iniciar o processamento, vamos importar os pacotes necessários.

In [1]:
!pip install openai==0.28.1

In [2]:
import re
import requests
from bs4 import BeautifulSoup
import openai

In [16]:
openai.api_key = input("Entre com a chave da API: ")

Entre com a chave da API: 


Faremos agora algumas funções de utilidade:

- `ask_chat_gpt` é recebe um `role` e um `prompt` e chama o endpoint *ChatCompletion* da OpenAI com o modelo `gpt-3.5-turbo`.
- `load_page` faz uma requisição a uma página via HTTP e retorna seu conteúdo HTML cru.

In [4]:
def ask_chat_gpt(role, prompt):
  msgs = [{
      'role': role,
      'content': prompt,
  }]

  res = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages = msgs,
  )

  return res.choices[0]["message"]["content"]

In [5]:
def load_page(url):
  headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:106.0) Gecko/20100101 Firefox/106.0"
  }

  res = requests.get(url, headers=headers)

  return res.content

Agora, iremos carregar os capítulos com os quais iremos trabalhar.

In [6]:
BOOK_URLS = [
    "https://brasileiraspln.com/livro-pln/1a-edicao/parte6/cap12/cap12.html",
    "https://brasileiraspln.com/livro-pln/1a-edicao/parte7/cap14/cap14.html",
]

In [7]:
page_contents = [ load_page(url) for url in BOOK_URLS ]

Note que, por estarmos restritos a um número de tokens no plano gratuito da OpenAI, não podemos ficar enviando o conteúdo completo dos capítulos em cada uma das requisições.

Portanto, iremos fazer um pré-processamento para separar apenas algum excertos dos capítulos. Daremos ao modelo que estamos usando a responsabilidade de escolher quais são as 2 seções mais importantes de cada capítulo, e rodaremos nossas análises apenas nelas.

Esse pré-processamento consiste em algumas funções que, quando compostas, nos darão a análise final que desejamos:

- `find_chapters`: descobre quais são as seções presentes no capítulo
- `get_chapter_content`: recebe o número da secão e retorna o conteúdo de texto da mesma
- `find_important_chapters`: envia os títulos das seções e pergunta ao modelo quais são os que ele julga mais importantes

In [8]:
def find_chapters(text):
  soup = BeautifulSoup(text)

  h3s = soup.select("h3")

  return [ h3.text.strip() for h3 in h3s ]

In [9]:
def get_chapter_content(text, chapter_idx):
  soup = BeautifulSoup(text)

  section = soup.select(f"section[data-number='{chapter_idx}']")

  section_texts = [ frag.text.strip() for frag in section ]

  return "\n".join(section_texts)

In [10]:
def find_important_chapters(chapters):
  chapter_list = "; ".join(chapters)

  prompt = f"Dada a lista de tópicos de um texto sobre PLN, escolha os 3 que julgar mais importantes: '{chapter_list}'. Responda apenas os números índices, separados por uma nova linha.";

  answer = ask_chat_gpt("user", prompt)

  return answer.split("\n")

Finalmente, a função `analyze_chapter` envia ao modelo o conteúdo da seção e pede ao modelo para fazer três análises:

- Correção gramatical;
- Sumarização;
- Extração de palavras-chave.

No `prompt` usamos um truque. Para evitar reenviar o conteúdo inteiro da seção para cada análise, gastando desnecessariamente tokens e banda de rede, mandamos os três comandos num mesmo prompt e instruímos o modelo a separar as respostas com um separador pré-definido -- neste caso, a cadeia `---SNIP---`. Com isso, podemos simplesmente separar as respostas com um `.split("---SNIP---")`.

A função irá retornar uma lista, com cada resposta em ordem.

In [11]:
def analyze_chapter(chapter_text):
  prompt = f"Considere o seguinte texto sobre PLN: \n\n'{chapter_text}'\n\nResponda corrigindo os erros de português do texto. Em seguida, escreva '---SNIP---' e sumarize o texto. Depois, escreva novamente '---SNIP---' e responda quais as palavras-chaves mais importantes do texto."

  answer = ask_chat_gpt("user", prompt)

  return answer.split("---SNIP---")

Agora, para cada capítulo estudado, vamos apenas compor as funções criadas e obter as análises.

In [13]:
for page_content in page_contents:
  chapters = find_chapters(page_content)
  important_chapters = find_important_chapters(chapters)

  for chapter in important_chapters:
    print(f"Analizando seção {chapter}...")

    chapter_content = get_chapter_content(page_content, chapter)

    analysis_result = analyze_chapter(chapter_content)

    print("Texto original:\n")
    print(chapter_content)
    print("-----------------------------\n");
    print("Texto corrigido:\n")
    print(analysis_result[0])
    print("-----------------------------\n");
    print("Texto resumido:\n");
    print(analysis_result[1])
    print("-----------------------------\n");
    print("Palavras-chave:\n")
    print(analysis_result[2])

Analizando seção 12.2.2...
Texto original:

12.2.2 Relações Semânticas Referenciais
Nesta subseção temos como foco tornar claro os tipos de relações semânticas que podem indicar uma relação de correferência.

12.2.2.1 Hiperonímia e Hiponímia
Hiperonímia é uma relação semântica que expressa um sentido amplo entre dois termos, partindo de uma classe mais ampla para uma subclasse mais específica, por exemplo: (inseto abelha). Neste caso, o termo “inseto” é um hiperônimo de “abelha”. Já Hiponímia representa uma relação contrária, ou seja, parte de uma classe mais específica para uma classe mais abrangente. Para o exemplo previamente dado temos que “abelha” é um hipônimo de “inseto”. Os hiperônimos e hipônimos são importantes no campo semântico, pois são muito usados na retomada de elementos em um texto, a fim de evitar repetições desnecessárias. No que diz respeito à identificação de menções referenciais em um discurso, na língua portuguesa é comum partirmos de termos específicos para term